In [1]:
%matplotlib inline
import backtrader as bt
import stockdata as sd
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

class MyStrategy(bt.Strategy):
    
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        self.datavolume = self.datas[0].volume
        
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        self.avg_volume = bt.indicators.SMA(self.datavolume, period=20)
        
    def next(self):
        if self.order:
            return
            
        if self.datavolume[0] > 3 * self.avg_volume[0]:
            self.order = self.buy()
                
        if self.order:
            return
            
        if self.buyprice is not None and (self.dataclose[0] < self.buyprice * 0.95 or self.dataclose[0] > self.buyprice * 1.3):
            self.order = self.sell()
            
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}')
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f'SELL EXECUTED, Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}')
                
            self.order = None
            
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            # self.log('Order Canceled/Margin/Rejected')
            print('Order Canceled/Margin/Rejected')
            
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        
        self.log(f'OPERATION PROFIT, GROSS {trade.pnl:.2f}, NET {trade.pnlcomm:.2f}')
        
cerebro = bt.Cerebro()

#bt.feeds.YahooFinanceData(dataname='AAPL', fromdate=datetime(2019, 1, 1), todate=datetime(2021, 1, 1))
df = sd.get_stock_price('000002')
df = df.sort_values('date')
df = df[df['date'] > '2022-12-01']
df

,Unnamed: 0,date,open,close,high,low,volumn,turnover,amp,percent_change,value_change,turnover_rate,open_qfq,close_qfq,high_qfq,low_qfq,open_org,close_org,high_org,low_org
日期,,,,,,,,,,,,,,,,,,,,
2022-12-02,2983,2022-12-02,18.50,18.39,18.74,17.78,1811997,3.304803e+09,5.05,-3.21,-0.61,1.86,18.50,18.39,18.74,17.78,18.50,18.39,18.74,17.78
2022-12-05,2984,2022-12-05,18.90,18.92,19.40,18.82,1937124,3.707004e+09,3.15,2.88,0.53,1.99,18.90,18.92,19.40,18.82,18.90,18.92,19.40,18.82
2022-12-06,2985,2022-12-06,18.80,19.18,19.26,18.45,1114424,2.120240e+09,4.28,1.37,0.26,1.15,18.80,19.18,19.26,18.45,18.80,19.18,19.26,18.45
2022-12-07,2986,2022-12-07,18.91,18.50,19.17,18.30,1396795,2.596530e+09,4.54,-3.55,-0.68,1.44,18.91,18.50,19.17,18.30,18.91,18.50,19.17,18.30
2022-12-08,2987,2022-12-08,18.55,19.38,19.50,18.53,1514868,2.900380e+09,5.24,4.76,0.88,1.56,18.55,19.38,19.50,18.53,18.55,19.38,19.50,18.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-06,61,2023-04-06,15.14,15.04,15.19,15.00,793381,1.193854e+09,1.25,-0.73,-0.11,0.82,15.14,15.04,15.19,15.00,15.14,15.04,15.19,15.00
2023-04-07,62,2023-04-07,15.28,15.82,15.83,15.11,2244567,3.505358e+09,4.79,5.19,0.78,2.31,15.28,15.82,15.83,15.11,15.28,15.82,15.83,15.11
2023-04-10,63,2023-04-10,15.75,15.70,15.82,15.58,1076086,1.688330e+09,1.52,-0.76,-0.12,1.11,15.75,15.70,15.82,15.58,15.75,15.70,15.82,15.58


In [2]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
data = bt.feeds.PandasData(dataname=df, datetime='date', open='open', high='high', low='low', close='close', volume='volumn', fromdate=datetime(2022, 1,1))
cerebro.adddata(data)
cerebro.addstrategy(MyStrategy)
cerebro.run()

plt.rcParams['figure.figsize'] = [15, 12]
plt.rcParams.update({'font.size': 12}) 
cerebro.plot(iplot=True)

<IPython.core.display.Javascript object>

ValueError: Image size of 1467x119274126 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1500x1200 with 4 Axes>